我们这里给二缺陷打上伪标签

In [ ]:
import sys
sys.path.append('..')

In [ ]:
from py_file.M_attri import Att
from py_file.Get_Data import DATA
from py_file.data_set import MyDataSet
from py_file.func_Test import Test_Func
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch

In [ ]:
trans = transforms.Resize(224)  # ResNet模型适合的图片大小为224x244
# 输入的张量需要带着批次维度和通道维度

# 加载数据和模型

## 获取数据

In [ ]:
train_data_path = '/mnt/workspace/DATA/train_WM.npz'
train_data = np.load(train_data_path)
print(train_data.files)

In [ ]:
ori_wm = train_data['original_wm']
wm = train_data['denoise_wm']
label = train_data['label_name']
label_oh = train_data['label_one_hot']
two_ori_wm = []
two_wm = []
two_label = []
two_label_oh = []
for i in range(len(label_oh)):
    if label_oh[i].sum() == 2:
        two_ori_wm.append(np.array(ori_wm[i]))
        two_wm.append(np.array(wm[i]))  # 元素是tensor的列表无法转换为张量，所以这里把列表里的元素转换为np.array
        two_label.append(label[i])
        two_label_oh.append(label_oh[i])

In [ ]:
two_wm_tensor = trans(torch.tensor(two_wm, dtype=torch.float32))
two_wm_tensor = torch.reshape(two_wm_tensor,(-1,1,224,224))  # 给图片加上通道
print(two_wm_tensor.shape)

## 获取类别名称和多热编码对应的字典

In [ ]:
two_label_unique, label_indices = np.unique(two_label, return_index=True)
print(two_label_unique)
print(label_indices)
# print(two_label_unique[indices])
two_label_oh_unique ,label_oh_indices = np.unique(two_label_oh,axis=0, return_index=True)
print(two_label_oh_unique)
print(label_oh_indices)

In [ ]:
oh_dict = {}
for index in label_indices:
    oh_dict[two_label[index]] = two_label_oh[index]
print(oh_dict)

## 加载模型和更新过的语义

In [ ]:
import pickle
model = torch.load('model_saved_pseudo/train_single.pth')
with open('updated_semantic_1_2/updated_single_dict.pkl', 'rb') as file:
    single_defect_att = pickle.load(file)
with open('updated_semantic_1_2/updated_two_dict.pkl', 'rb') as file:
    two_defect_att = pickle.load(file)

In [ ]:
two_att = []
for l in two_label:
    two_att.append(two_defect_att[l])
two_att_tensor = torch.tensor(np.array(two_att), dtype=torch.float32)
print(two_att_tensor.shape)

In [ ]:
two_dataset = MyDataSet(two_wm_tensor, two_att_tensor)
two_loader = DataLoader(two_dataset, batch_size=32, shuffle=False)

# 定义预测标签的函数

In [ ]:
from py_file.func_Test import Test_Func
# 需要的函数都已经集成在了Test_Func里
func = Test_Func()

# 开始打伪标签

In [ ]:
real_label, predict_label = func.get_label(model, two_loader, two_defect_att, 'cos')

In [ ]:
real_label == two_label

In [ ]:
two_label = np.array(two_label)
predict_label = np.array(predict_label)
print(two_label.shape, predict_label.shape)

In [ ]:
predict_two_label_oh = []
for l in predict_label:
    predict_two_label_oh.append(oh_dict[l])
    
predict_two_label_oh = np.array(predict_two_label_oh)
print(predict_two_label_oh.shape)

# 保存打了伪标签的二故障样本

In [ ]:
two_ori_wm = np.array(two_ori_wm)
two_wm = np.array(two_wm)
two_label_oh = np.array(two_label_oh)
print(two_ori_wm.shape)
print(two_wm.shape)
print(two_label_oh.shape)

In [ ]:
np.savez('data_fake_label/two_fake_label_WM.npz', original_wm = two_ori_wm, label_one_hot = predict_two_label_oh,
 denoise_wm = two_wm, label_name =predict_label)